In [1]:
import socket
import sys

is_kaggle = False
if not is_kaggle:
    from my_utils import get_notebook_path
    get_notebook_path().split('/')[-1].split('.')[0]
    NB = get_notebook_path().split('/')[-1].split('.')[0]
    HOST = socket.gethostname()
    import mlflow
    from logzero import logger
    DELIMITER = '\\'

else:
    NB = 'exp0001'
    HOST = 'DESKTOP-M3SEAIN'
    sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
    DELIMITER = '/'
description = '256'

print(f'{HOST}_{NB}')

NB, HOST

DESKTOP-M3SEAIN_exp0100_embeding


('exp0100_embeding', 'DESKTOP-M3SEAIN')

In [2]:
#from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from sklearn.model_selection import StratifiedKFold, KFold
if not is_kaggle:
    from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, f1_score
from pathlib import Path
from tqdm import tqdm
from collections import OrderedDict
import random
import os
import gc
import shutil
import timm


import torch.optim as optim
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import albumentations as albu

import pickle

import warnings
warnings.simplefilter('ignore')

if is_kaggle:
    DATA_DIR = Path('../input/petfinder-pawpularity-score')
    OUTPUT_DIR = Path('./')
    CP_DIR = Path('../input') / Path('DESKTOP-M3SEAIN_exp0001'.lower().replace('-', '').replace('_', '-'))
else:
    ROOT_DIR = Path('../')
    DATA_DIR = ROOT_DIR / 'data'
    OUTPUT_DIR = Path('../') / 'output'
    CP_DIR = OUTPUT_DIR / 'checkpoint'

def to_pickle(filename, obj):
    with open(filename, mode='wb') as f:
        pickle.dump(obj, f)
        
def unpickle(filename):
    with open(filename, mode='rb') as fo:
        p = pickle.load(fo)
    return p 

C:\Users\owner\.conda\envs\kaggle\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
class Config:
    N_LABEL = 4
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 1.0e-05
    MAX_LR = 8.0e-4
    PATIENCE = 18
    EPOCH = 50
    BATCH_SIZE = 4
    SKIP_EVALUATE_NUM = 0
    BACK_BONE = 'tf_efficientnet_b0_ns'
    RUN_FOLD_COUNT = 10
    IMG_SIZE=256
    T_MAX=20
    ETA_MIN=3.0e-7
    SCHEDULER_GAMMA=1.0
    ACCUMULATION_STEMP=2
    
if is_kaggle:
    Config.BATCH_SIZE = 6
    
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(seed=Config.RANDOM_SATE)

In [4]:
#print(timm.list_models(pretrained=True))

In [5]:
train_labels = np.load('../data/train_data.npz')['arr_1']
train_imgs = np.load('../data/train_data.npz')['arr_0']
test_imgs = np.load('../data/test_data.npz')['arr_0']

In [6]:
train_labels_df = pd.get_dummies(pd.DataFrame(train_labels)[0])

In [7]:
def get_augmentation():
    train_transform = [
        albu.Resize(height=Config.IMG_SIZE, width=Config.IMG_SIZE),
        albu.Normalize(),
    ]
    return albu.Compose(train_transform)

In [8]:
class EmbDataSet(Dataset):
    def __init__(self, images, transforms, data_type=None):
        self.images = images
        self.data_type = data_type
        self.transforms = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        
        augmented = self.transforms(image=img)
        img = augmented['image']
        img = np.moveaxis(img, 2, 0)
        
        return img


In [9]:
class EmbNet(nn.Module):
    def __init__(self):
        super(EmbNet, self).__init__()

        self.base_model = timm.create_model(Config.BACK_BONE, num_classes=0, pretrained=True, in_chans=3)
        num_features = self.base_model.num_features

        self.cls = nn.Sequential(
            nn.Linear(num_features, int(num_features / 2)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features / 2), int(num_features / 4)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features / 4), Config.N_LABEL)
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.cls(x)
        return x

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
%%time

net = EmbNet().base_model
net.to(device)

train_outputs = []
test_outputs = []
train_dataset = EmbDataSet(train_imgs, get_augmentation(), data_type='train')
test_dataset = EmbDataSet(test_imgs, get_augmentation(), data_type='test')

trainloader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, shuffle=False, drop_last=False)
testloader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, shuffle=False, drop_last=False)

for img in tqdm(trainloader, total=len(trainloader)):
    net.eval()

    with torch.no_grad():
        img = img.to(device).float()
        output = net(img)
        output = output.to('cpu').detach().numpy().copy()
        train_outputs.append(output)
        
for img in tqdm(testloader, total=len(testloader)):
    net.eval()

    with torch.no_grad():
        img = img.to(device).float()
        output = net(img)
        output = output.to('cpu').detach().numpy().copy()
        test_outputs.append(output)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 34.61it/s]


Wall time: 1min 6s


In [14]:
train_emb = np.vstack(train_outputs)
test_emb = np.vstack(test_outputs)
train_emb

array([[ 0.05335956,  0.04720343,  0.30871895, ...,  0.36370742,
         0.75598   ,  0.5224472 ],
       [-0.00957962, -0.11519627, -0.19632664, ...,  0.26870325,
        -0.19640139, -0.14268734],
       [ 0.02004881,  0.46732742,  0.09173357, ...,  0.34954405,
        -0.00622322,  0.38592005],
       ...,
       [-0.09880641,  0.1422289 ,  0.3452528 , ...,  0.04504676,
         0.26873484,  0.03510376],
       [ 0.00991592, -0.12312946, -0.15985121, ...,  0.5627369 ,
         0.8107333 , -0.08827448],
       [-0.08532782,  0.22903477,  0.35048908, ...,  0.61091995,
        -0.0555803 , -0.01716992]], dtype=float32)